## Factors

原資産と時系列データから得られる数値をFactor、Factorを算出する関数をFactor関数と言います。

```
F(asset, timestamp) -> float
```

Factorの例として、移動平均があります。`SimpleMovingAverage` は移動平均を算出するFactor関数です。引数には対象となる銘柄の時系列データに加えてキーワード引数 `window_length` に算出する期間を渡します。
株価のデータは `USEquityPricing` から得られます。

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# New from the last lesson, import the USEquityPricing dataset.
from quantopian.pipeline.data.builtin import USEquityPricing

# New from the last lesson, import the built-in SimpleMovingAverage factor.
from quantopian.pipeline.factors import SimpleMovingAverage

### Creating a Factor

`SimpleMovingAverage` factorを作成してみます。キーワード引数 `inputs` には `BoundColumn` オブジェクト(後述)、キーワード引数 `window_length` には期間の個数を整数値で渡します。

下記のコードでは10日間の移動平均を算出するためのfactorを作成します。

In [2]:
mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)

factorの作成では計算は実行されません。関数を定義した段階では関数が実行されないことと同義です。factorはpiplineに追加して実行されます。

### Adding a Factor to a Pipeline

空のpipelineにインスタンス化した移動平均のfactorを追加します。`Pipeline()` 関数のキーワード引数 `columns` に列名をキーとし、値をfanctorインスタンスを渡すことでpipelineにfactorが追加されます。

In [3]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    
    return Pipeline(
        columns={
            '10_day_mean_close': mean_close_10
        }
    )

実際にpipelineインスタンスを作成し、実行してみます。

In [4]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

10_day_mean_close
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])                13.5595
                          Equity(21 [AAME])                3.9625
                          Equity(24 [AAPL])              129.0257
                          Equity(25 [ARNC_PR])            88.3625
                          Equity(31 [ABAX])               61.9209

8000以上の銘柄の10日間のSMAが列に追加されました。indexはMultiIndexとなっており、level=0には日付、level=1には銘柄となっています。

piplineを複数日を与えて実行すると下記のようになります。

In [5]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-07')
result.head()

10_day_mean_close
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])                13.5595
                          Equity(21 [AAME])                3.9625
                          Equity(24 [AAPL])              129.0257
                          Equity(25 [ARNC_PR])            88.3625
                          Equity(31 [ABAX])               61.9209

In [6]:
result.tail()

10_day_mean_close
2015-05-07 00:00:00+00:00 Equity(49009 [PRKU])                NaN
                          Equity(49010 [TBRA])                NaN
                          Equity(49011 [CCR])                 NaN
                          Equity(49012 [FXEU])                NaN
                          Equity(49013 [TEGP])                NaN

Note: factors can also be added to an existing `Pipeline` instance using the `Pipeline.add` method. Using `add` looks something like this:
    >>> my_pipe = Pipeline()
    >>> f1 = SomeFactor(...)
    >>> my_pipe.add(f1)

作成済みのpiplineのインスタンスには下記のように factorを追加することができます。

```
>>> my_pipe = Pipeline()
>>> f1 = SomeFactor(...)
>>> my_pipe.add(f1)
```

### Latest

`Latest` は最もよく使われる組み込みfactorです。`Latest` はデータ列から最新の値を取得します。`.latest` 属性を参照する方法がよく使われます。

In [7]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    latest_close = USEquityPricing.close.latest

    return Pipeline(
        columns={
            '10_day_mean_close': mean_close_10,
            'latest_close_price': latest_close
        }
    )

10日間の移動平均に加え、直近の終値が追加されています。

In [8]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head(5)

10_day_mean_close  \
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])                13.5595   
                          Equity(21 [AAME])                3.9625   
                          Equity(24 [AAPL])              129.0257   
                          Equity(25 [ARNC_PR])            88.3625   
                          Equity(31 [ABAX])               61.9209   

                                                latest_close_price  
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])                  14.015  
                          Equity(21 [AAME])                    NaN  
                          Equity(24 [AAPL])                128.699  
                          Equity(25 [ARNC_PR])                 NaN  
                          Equity(31 [ABAX])                 55.030

## Default Inputs

デフォルトで入力されているfactorは、変更してはいけません。例えば [VWAP built-in factor](https://www.quantopian.com/help#built-in-factors) は `USEquityPricing.close` と `USEquityPricing.volume` から算出されます。常に同じ `BoundColumns` からfactorが算出される場合、入力を指定せずにコンストラクタを呼び出すことができます。

In [9]:
from quantopian.pipeline.factors import VWAP
vwap = VWAP(window_length=10)

In the next lesson, we will look at combining factors.

次のレッスンではfactor同士を組み合わせます。